In [147]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [41]:
%sendtofile --source UDPblackviewphone.py
%sendtofile --source ubxminimal.py

Sent 66 lines (2004 bytes) to ubxminimal.py.


In [42]:
%sendtofile deviceletter.txt
C

Sent 1 lines (1 bytes) to deviceletter.txt.


In [181]:
%sendtofile main.py

import time, machine, urandom

# delay long enough to interrupt if we need to, 
# before the uart or wlan bricks it for debugging
pled = machine.Pin(2, machine.Pin.OUT)
for i in range(11):
    pled.value(i%2)
    time.sleep_ms(500)  


from ubxminimal import uartUBX, initUBX
initUBX()

for i in range(11):
    pled.value(i%2)
    time.sleep_ms(200)


deviceletter = open("deviceletter.txt", "r").read()
port = {"A":9020, "B":9021, "C":9022}[deviceletter]
udpaddr = ("192.168.43.1", port)  # ip default for android

from UDPblackviewphone import connectBlackview, connectS5
if deviceletter == 'C':
    connectS5(pled)
else:
    connectBlackview(pled)

for i in range(11):
    pled.value(i%2)
    time.sleep_ms(100)


import socket
s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
s.settimeout(0.005)
    
ubxbuffer = bytearray(1000)
mubxbuffer = memoryview(ubxbuffer)

# main loop
PCudpaddr = None
reccount = 0
while True:
    n = uartUBX.readinto(mubxbuffer[2:])
    if n is not None:
        ubxbuffer[0] = reccount&0xFF
        ubxbuffer[1] = (reccount>>8)&0xFF
        reccount += 1
        try:
            s.sendto(mubxbuffer[:n+2], udpaddr)
            if PCudpaddr is not None:
                s.sendto(mubxbuffer[:n+2], PCudpaddr)
        except OSError as e:
            print("dwrite", e)
        #print(x[:5])
        pled.value(1-pled.value())
    else:
        try:
            mess, PCudpaddr = s.recvfrom(50)
            print(mess, PCudpaddr)
        except OSError as e:
            pass # timeout



Sent 64 lines (1521 bytes) to main.py.


In [163]:

from ubxminimal import uartUBX, initUBX
initUBX()


.

In [168]:
ubxbuffer = bytearray(1000)
mubxbuffer = memoryview(ubxbuffer)
mubxbuffer2 = mubxbuffer[2:]

In [180]:
ubxbuffer[0] = 0xff
for i in range(50):
    k = uartUBX.readinto(mubxbuffer[2:])
    print(i, k)

0 328
1 None
2 None
3 None
4 None
5 None
6 None
7 None
8 None
9 None
10 None
11 None
12 None
13 None
14 None
15 None
16 None
17 None
18 None
19 120
20 None
21 None
22 None
23 None
24 None
25 None
26 38
27 None
28 None
29 None
30 None
31 None
32 None
33 None
34 None
35 None
36 None
37 None
38 None
39 None
40 None
41 None
42 None
43 None
44 None
45 None
46 None
47 None
48 None
49 None


In [51]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [34]:
import os
print(os.listdir())

['boot.py', 'ubxutils.py', 'UDPsocket_funcs.py', 'main.py', 'UDPblackviewphone.py', 'ubxminimal.py', 'deviceletter.txt']


In [35]:
deviceletter = open("deviceletter.txt", "r").read()
print([deviceletter])

['B']


In [52]:

# test UBX
import machine
uartUBX = machine.UART(1, baudrate=9600, rx=16, tx=17)
print(uartUBX.read())

None


In [55]:
for l in uartUBX:
    print(l)

b',99.99,,,,,,*77\r\n'
b'$GNGSA,A,1,,,,,,,,,,,,,99.99,99.99,99.99*2E\r\n'
b'$GNGSA,A,1,,,,,,,,,,,,,99.99,99.99,99.99*2E\r\n'
b'$GPGSV,4,1,14,01,07,005,12,10,29,277,09,12,51,206,13,13,07,142,*7B\r\n'
b'$GPGSV,4,2,14,14,03,314,,15,33,165,,17,30,055,08,18,01,339,*70\r\n'
b'$GPGSV,4,3,14,19,35,086,,20,18,248,19,24,83,110,,25,17,222,08*70\r\n'
b'$GPGSV,4,4,14,28,00,047,16,32,17,312,*75\r\n'
b'$GLGSV,3,1,0920,18,248,08,24,83,110,,25,17,222,11*71\r\n'
b'$GPGSV,4,4,14,28,00,047,08,32,17,312,*7A\r\n'
b'$GLGSV,3,1,09,66,35,202,,67,55,281,,6'
